In [2]:
# Práctica 6: ML US

import keras
import numpy as np
import os
import skimage
from PIL import Image
from skimage import measure
import cv2
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from matplotlib import pyplot as plt
from metricas import *

In [3]:
# LAS SIGUIENTES ÓRDENES GUARDAN UN LISTADO DE LAS IMÁGENES DE LOS DIRECTORIOS ORIGINAL Y MÁSCARA. DEBÉIS REPETIRLO PARA TODOS LOS DIRECTORIOS QUE TENÉIS EN P6_IA

# Training benign
path_training_benign_original='benign/training/original'
lista_ficheros1=os.listdir(path_training_benign_original)
lista_ficheros1.sort()

path_training_benign_mask='benign/training/mask'
lista_ficheros2=os.listdir(path_training_benign_mask)
lista_ficheros2.sort()

# Test benign
path_test_benign_original='benign/test/original'
lista_ficheros3=os.listdir(path_test_benign_original)
lista_ficheros3.sort()

path_test_benign_mask='benign/test/mask'
lista_ficheros4=os.listdir(path_test_benign_mask)
lista_ficheros4.sort()


# Training malignant
path_training_malignant_original='malignant/training/original'
lista_ficheros5=os.listdir(path_training_malignant_original)
lista_ficheros5.sort()

path_training_malignant_mask='malignant/training/mask'
lista_ficheros6=os.listdir(path_training_malignant_mask)
lista_ficheros6.sort()

# Test malignant
path_test_malignant_original='malignant/test/original'
lista_ficheros7=os.listdir(path_test_malignant_original)
lista_ficheros7.sort()

path_test_malignant_mask='malignant/test/mask'
lista_ficheros8=os.listdir(path_test_malignant_mask)
lista_ficheros8.sort()


# Extracción y selección de características

## En primer lugar extraemos las características seleccionadas

### Extracción características para entrenamiento

In [4]:
# num_caract: número de caracteríticas
# Existen 40 casos en total de entrenamiento (20 de tumores malignos y 20 benignos)

# En cada fila una imagen, en cada columna una caracteristica
num_caract = 12
features=np.zeros((40,num_caract))  

# Imágenes de entrenamiento tumor benigno
i=0
for image,image_mask in zip(lista_ficheros1,lista_ficheros2):
    I=Image.open(os.path.join(path_training_benign_original,image))
    I_mask=Image.open(os.path.join(path_training_benign_mask,image_mask))

    I = np.array(I.convert('L')); I_mask = measure.label(np.array(I_mask.convert('L')))
    props = measure.regionprops(I_mask,I)
    
    # Características seleccionadas
    features[i][0] = props[0].perimeter**2/props[0].area
    features[i][1] = props[0].eccentricity
    features[i][2] = props[0].solidity
    features[i][3] = np.mean(I[np.where(I_mask==1)])
    features[i][4] = props[0].weighted_moments_hu[0]
    features[i][5] = props[0].weighted_moments_hu[1]
    features[i][6] = props[0].weighted_moments_hu[2]
    features[i][7] = props[0].weighted_moments_hu[3]
    features[i][8] = props[0].weighted_moments_hu[4]
    features[i][9] = props[0].weighted_moments_hu[5]
    features[i][10] = props[0].weighted_moments_hu[6]
    features[i][11] = props[0].euler_number    

    i = i+1

# Imágenes de entrenamiento tumor maligno
for image,image_mask in zip(lista_ficheros5,lista_ficheros6):
    I=Image.open(os.path.join(path_training_malignant_original,image))
    I_mask=Image.open(os.path.join(path_training_malignant_mask,image_mask))

    I = np.array(I.convert('L')); I_mask = measure.label(np.array(I_mask.convert('L')))
    props = measure.regionprops(I_mask,I)
    
    # Características seleccionadas
    features[i][0] = props[0].perimeter**2/props[0].area
    features[i][1] = props[0].eccentricity
    features[i][2] = props[0].solidity  
    features[i][3] = np.mean(I[np.where(I_mask==1)])
    features[i][4] = props[0].weighted_moments_hu[0]
    features[i][5] = props[0].weighted_moments_hu[1]
    features[i][6] = props[0].weighted_moments_hu[2]
    features[i][7] = props[0].weighted_moments_hu[3]
    features[i][8] = props[0].weighted_moments_hu[4]
    features[i][9] = props[0].weighted_moments_hu[5]
    features[i][10] = props[0].weighted_moments_hu[6]   
    features[i][11] = props[0].euler_number
    
    i = i+1

# Normalizamos cada columna (característica) restando su mínimo y dividiendo por su máximo  
f_min = features-np.amin(features,0)
features_norm = f_min/np.amax(f_min,0)

# Aleatorizamos para mejorar el entrenamiento de la red neuronal
np.random.seed(72)
idx = np.random.permutation(len(features))
features_norm = features_norm[idx,:]

# GENERACIÓN DE LA VERDAD DE REFERENCIA BASADO EN 'ONE-HOT ENCONDING'.
# Verdad de referencia de las imágenes de entrenamiento
GT = np.zeros((40,2))
GT[0:20,0]  = 1   
GT[20:40,1] = 1

# Aleatorizamos de la misma forma que las características de entrenamiento para conservar la correspondencia
GT = GT[idx,:]


### Extracción de características para test

In [5]:
# Repetir la extracción de características de las imágenes de entrenamiento para las imágenes de test
features_test = np.zeros((20,num_caract))  

# Imágenes de test tumor benigno
i = 0
for image,image_mask in zip(lista_ficheros3,lista_ficheros4):
    
    I=Image.open(os.path.join(path_test_benign_original,image))
    I_mask=Image.open(os.path.join(path_test_benign_mask,image_mask))

    I = np.array(I.convert('L')); I_mask = measure.label(np.array(I_mask.convert('L')))
    props = measure.regionprops(I_mask,I)
    
    # Características seleccionadas
    features_test[i][0] = props[0].perimeter**2/props[0].area
    features_test[i][1] = props[0].eccentricity
    features_test[i][2] = props[0].solidity
    features_test[i][3] = np.mean(I[np.where(I_mask==1)])
    features_test[i][4] = props[0].weighted_moments_hu[0]
    features_test[i][5] = props[0].weighted_moments_hu[1]
    features_test[i][6] = props[0].weighted_moments_hu[2]
    features_test[i][7] = props[0].weighted_moments_hu[3]
    features_test[i][8] = props[0].weighted_moments_hu[4]
    features_test[i][9] = props[0].weighted_moments_hu[5]
    features_test[i][10] = props[0].weighted_moments_hu[6]
    features_test[i][11] = props[0].euler_number

    i = i+1

# Imágenes de test tumor maligno
for image,image_mask in zip(lista_ficheros7,lista_ficheros8):
    
    I=Image.open(os.path.join(path_test_malignant_original,image))
    I_mask=Image.open(os.path.join(path_test_malignant_mask,image_mask))

    I = np.array(I.convert('L')); I_mask = measure.label(np.array(I_mask.convert('L')))
    props = measure.regionprops(I_mask,I)
    
    # Características seleccionadas
    features_test[i][0] = props[0].perimeter**2/props[0].area
    features_test[i][1] = props[0].eccentricity
    features_test[i][2] = props[0].solidity  
    features_test[i][3] = np.mean(I[np.where(I_mask==1)])
    features_test[i][4] = props[0].weighted_moments_hu[0]
    features_test[i][5] = props[0].weighted_moments_hu[1]
    features_test[i][6] = props[0].weighted_moments_hu[2]
    features_test[i][7] = props[0].weighted_moments_hu[3]
    features_test[i][8] = props[0].weighted_moments_hu[4]
    features_test[i][9] = props[0].weighted_moments_hu[5]
    features_test[i][10] = props[0].weighted_moments_hu[6]
    features_test[i][11] = props[0].euler_number

    i=i+1  

# Normalizamos cada columna (característica) restando su mínimo y dividiendo por su máximo  
f_min_test = features_test-np.amin(features_test,0)
features_test_norm = f_min_test/np.amax(f_min_test,0)

# GENERACIÓN DE LA VERDAD DE REFERENCIA DE LAS IMÁGENES DE TEST
GT_test = np.zeros((20,2))
GT_test[0:10,0]  = 1   
GT_test[10:20,1] = 1

# Para las imágenes de test no es necesario aleatorizar su orden pues éste no influye en nada

## Selección de características

In [6]:
# Flag para indicar que se ha llevado a cabo la selección de características
seleccion = False 

# Para selección de características empleamos SelectKBest de sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Selección de características: de las 12 que tenemos a "k" características
k = 8 # Número de características que quedarán tras la selección
test = SelectKBest(score_func=chi2, k=k)
fit = test.fit(features_norm, GT)
np.set_printoptions(precision=3)

# Transformación del conjunto de características de entrenamiento y de test para quedarnos con las 
# "k" mejores características obtenidas en el paso anterior
features_selec = fit.transform(features_norm)
features_test_selec = fit.transform(features_test_norm)

# Clasificador Red neuronal 

In [7]:
# Destinar parte del conjunto de entrenamiento a validacion: un 20%, 8 imagenes para validación

# Según si ha habido o no selección de características empleamos un conjunto de características u otro
if seleccion:
    # Separar conjuntos de entrenamiento y validación
    features_norm_train = features_selec[:-8,:]
    features_norm_val = features_selec[32:,:]
    input_carat = features_selec.shape[1]
    
else:
    # Separar conjuntos de entrenamiento y validación
    features_norm_train = features_norm[:-8,:]
    features_norm_val = features_norm[32:,:]
    input_carat = features_norm_train.shape[1]    

# Separamos la verdad de referencia para entrenamiento y para validación para que se corresponda a los 
# conjuntos de características de entrenamiento y validación separados en el paso anterior
GT_train = GT[:-8,:]
GT_val = GT[32:,:]

# CLASIFICADOR RED NEURONAL : input_caract NEURONAS DE ENTRADA, TANTAS COMO CARACTERÍSTICAS
#                             256 NEURONAS CAPA OCULTA
#                             2  NEURONAS DE SALIDA, UNA POR CLASE)

model = Sequential()
model.add(Dense(256, input_shape=(input_carat,)))
model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(optimizer= 'adam', loss='kld', metrics=["accuracy"])

# Entrenamos la red con los conjuntos de entrenamiento y validación
model.fit(features_norm_train, GT_train, epochs = 200, validation_data=(features_norm_val, GT_val))


2023-01-23 22:42:00.674097: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-23 22:42:00.674236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-23 22:42:01.010891: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-01-23 22:42:01.011088: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/200
1/1 [==============================] - ETA: 0s - loss: 0.7225 - accuracy: 0.4688

2023-01-23 22:42:07.166757: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 7s 7s/step - loss: 0.7225 - accuracy: 0.4688 - val_loss: 0.6725 - val_accuracy: 0.6250
Epoch 2/200
1/1 [==============================] - 0s 22ms/step - loss: 0.6897 - accuracy: 0.5000 - val_loss: 0.6989 - val_accuracy: 0.3750
Epoch 3/200
1/1 [==============================] - 0s 21ms/step - loss: 0.6768 - accuracy: 0.5312 - val_loss: 0.7322 - val_accuracy: 0.3750
Epoch 4/200
1/1 [==============================] - 0s 21ms/step - loss: 0.6787 - accuracy: 0.5312 - val_loss: 0.7556 - val_accuracy: 0.3750
Epoch 5/200
1/1 [==============================] - 0s 21ms/step - loss: 0.6841 - accuracy: 0.5312 - val_loss: 0.7608 - val_accuracy: 0.3750
Epoch 6/200
1/1 [==============================] - 0s 22ms/step - loss: 0.6848 - accuracy: 0.5312 - val_loss: 0.7508 - val_accuracy: 0.3750
Epoch 7/200
1/1 [==============================] - 0s 19ms/step - loss: 0.6802 - accuracy: 0.5312 - val_loss: 0.7323 - val_accuracy: 0.3750
Epoch 8/200
1/1 [=================

2023-01-23 22:42:07.499893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 21ms/step - loss: 0.6732 - accuracy: 0.5312 - val_loss: 0.7113 - val_accuracy: 0.3750
Epoch 9/200
1/1 [==============================] - 0s 20ms/step - loss: 0.6669 - accuracy: 0.5312 - val_loss: 0.6919 - val_accuracy: 0.3750
Epoch 10/200
1/1 [==============================] - 0s 19ms/step - loss: 0.6631 - accuracy: 0.5312 - val_loss: 0.6768 - val_accuracy: 0.8750
Epoch 11/200
1/1 [==============================] - 0s 19ms/step - loss: 0.6620 - accuracy: 0.9688 - val_loss: 0.6665 - val_accuracy: 0.6250
Epoch 12/200
1/1 [==============================] - 0s 18ms/step - loss: 0.6623 - accuracy: 0.8125 - val_loss: 0.6605 - val_accuracy: 0.6250
Epoch 13/200
1/1 [==============================] - 0s 19ms/step - loss: 0.6622 - accuracy: 0.5625 - val_loss: 0.6579 - val_accuracy: 0.6250
Epoch 14/200
1/1 [==============================] - 0s 19ms/step - loss: 0.6607 - accuracy: 0.5625 - val_loss: 0.6579 - val_accuracy: 0.6250
Epoch 15/200
1/1 [=========

## Predicción con las características de las imágenes de test

In [9]:
# Para evitar que imprima los warning 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

# Según si ha habido o no selección de características empleamos un conjunto de características para test u otro
if seleccion:
    # Realizamos la clasificacion de las imagenes de test
    pred = model.predict(features_test_selec)
    
else:
    # Realizamos la clasificacion de las imagenes de test
    pred = model.predict(features_test_norm)

# Pasamos las probabilidades a clases: 0 -> Benigno, 1 -> Maligno
clasificacion = np.argmax(pred,axis=1)

# Resultados de la predicción con la red neuronal
Mt_confusion = matriz_confusion(clasificacion,GT_test)
metricas(Mt_confusion)

Matriz de confusion: [[10.  0.]
 [ 0. 10.]]

Accuracy: 1.0
Sensibilidad: 1.0
Especificidad: 1.0
F1 Score: 1.0
Valor Predictivo Positivo: 1.0
Valor Predictivo Negativo: 1.0


2023-01-23 22:40:30.809665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


# Clasificador SVM

In [10]:
# Clasificación con SVM: Usamos NuSVC, uno de los modelos disponibles
from sklearn.feature_selection import SelectFromModel
from sklearn import svm
clf = svm.NuSVC(gamma="auto")

# Según si ha habido o no selección de características empleamos un conjunto de características para test u otro
if seleccion:   
    # Entrenamos el modelo. Solo admite una columna de GT
    clf.fit(features_selec, GT[:,1])
    # Predicción
    pred_SVM = clf.predict(features_test_selec)
    
else:
    # Entrenamos el modelo. Solo admite una columna de GT
    clf.fit(features_norm, GT[:,1])
    # Predicción
    pred_SVM = clf.predict(features_test_norm)
    
# Métricas de resultado
Mt_confusion_SVM = matriz_confusion(pred_SVM,GT_test)
metricas(Mt_confusion_SVM)

Matriz de confusion: [[9. 1.]
 [1. 9.]]

Accuracy: 0.9
Sensibilidad: 0.9
Especificidad: 0.9
F1 Score: 0.9
Valor Predictivo Positivo: 0.9
Valor Predictivo Negativo: 0.9


# Clasificador SGD: Descenso Estocástico de Gradiente

In [11]:
# Importamos el modelo
from sklearn.linear_model import SGDClassifier
SGD_Clasificador = SGDClassifier(loss="modified_huber", penalty="elasticnet", max_iter=19)

# Según si ha habido o no selección de características empleamos un conjunto de características para test u otro
if seleccion:   
    # Entrenamos el modelo. Solo admite una columna de GT
    SGD_Clasificador.fit(features_selec, GT[:,1])
    # Predicción
    pred_SGD = SGD_Clasificador.predict(features_test_selec)
    
else:
    # Entrenamos el modelo. Solo admite una columna de GT
    SGD_Clasificador.fit(features_norm, GT[:,1])
    # Predicción
    pred_SGD = SGD_Clasificador.predict(features_test_norm)

# Métricas de resultado
Mt_confusion_SGD = matriz_confusion(pred_SGD,GT_test)
metricas(Mt_confusion_SGD)

Matriz de confusion: [[9. 1.]
 [1. 9.]]

Accuracy: 0.9
Sensibilidad: 0.9
Especificidad: 0.9
F1 Score: 0.9
Valor Predictivo Positivo: 0.9
Valor Predictivo Negativo: 0.9


# Clasificador Nearest Centroid Classifier

In [12]:
# Importamos el modelo
from sklearn.neighbors import NearestCentroid
NC = NearestCentroid()

# Según si ha habido o no selección de características empleamos un conjunto de características para test u otro
if seleccion:   
    # Entrenamos el modelo. Solo admite una columna de GT
    NC.fit(features_selec, GT[:,1])
    # Predicción
    pred_NC = NC.predict(features_test_selec)
    
else:
    # Entrenamos el modelo. Solo admite una columna de GT
    NC.fit(features_norm, GT[:,1])
    # Predicción
    pred_NC = NC.predict(features_test_norm)

# Métricas de resultado
Mt_confusion_NC = matriz_confusion(pred_NC,GT_test)
metricas(Mt_confusion_NC)

Matriz de confusion: [[10.  1.]
 [ 0.  9.]]

Accuracy: 0.95
Sensibilidad: 1.0
Especificidad: 0.9
F1 Score: 0.9523809523809523
Valor Predictivo Positivo: 0.9090909090909091
Valor Predictivo Negativo: 1.0
